In [ ]:
!pip install Pillow==9.0.0

TODO: Restart the RunTime before running the following cells

In [ ]:
! pip install -q mmcv mmaction einops wget timm tensorflow-addons

In [ ]:
import sys
import os

import tensorflow as tf
from typing import List


Convert PyTorch Weights to TF weights

In [ ]:
! git clone https://github.com/shoaib6174/GSOC-22-Video-Swin-Transformers
sys.path.append("/content/GSOC-22-Video-Swin-Transformers")

In [ ]:
from VideoSwinTransformer import model_configs

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [4]:
for model_name in model_configs.MODEL_MAP:
  print("Converting", model_name)
  command = f"python /content/GSOC-22-Video-Swin-Transformers/convert.py -m {model_name}"
  os.system(command)

Convert weights' files to tar.gz

In [ ]:
TF_MODEL_ROOT = "tf_models/"
TAR_ARCHIVES = os.path.join("tars/")


def prepare_archive(model_name: str) -> None:
    """Prepares a tar archive."""
    archive_name = f"{model_name}.tar.gz"
    print(f"Archiving to {archive_name}.")
    archive_command = f"cd {model_name} && tar -czvf ../{archive_name} *"
    os.system(archive_command)
    os.system(f"rm -rf {model_name}")


def save_to_gcs(model_paths: List[str]) -> None:
    """Prepares tar archives and saves them inside a GCS bucket."""
    for path in model_paths:
        print(f"Preparing model: {path}.")
        model_name = path.strip("/")
        abs_model_path = os.path.join(TF_MODEL_ROOT, model_name)

        print(f"Copying from {abs_model_path}.")
        os.system(f"gsutil cp -r {abs_model_path} .")
        prepare_archive(model_name)

    os.system(f"gsutil -m cp -r *.tar.gz {TAR_ARCHIVES}")
    os.system("rm -rf *.tar.gz")

In [ ]:
model_paths = tf.io.gfile.listdir(TF_MODEL_ROOT)
print(f"Total models: {len(model_paths)}.")

print("Preparing archives for the classification and feature extractor models.")
save_to_gcs(model_paths)
tar_paths = tf.io.gfile.listdir(TAR_ARCHIVES)
print(f"Total tars: {len(tar_paths)}.")

Total models: 7.
Preparing archives for the classification and feature extractor models.
Preparing model: swin_base_patch244_window877_kinetics600_22k_tf.
Copying from tf_models/swin_base_patch244_window877_kinetics600_22k_tf.
Archiving to swin_base_patch244_window877_kinetics600_22k_tf.tar.gz.
Preparing model: swin_small_patch244_window877_kinetics400_1k_tf.
Copying from tf_models/swin_small_patch244_window877_kinetics400_1k_tf.
Archiving to swin_small_patch244_window877_kinetics400_1k_tf.tar.gz.
Preparing model: swin_base_patch244_window1677_sthv2_tf.
Copying from tf_models/swin_base_patch244_window1677_sthv2_tf.
Archiving to swin_base_patch244_window1677_sthv2_tf.tar.gz.
Preparing model: swin_base_patch244_window877_kinetics400_1k_tf.
Copying from tf_models/swin_base_patch244_window877_kinetics400_1k_tf.
Archiving to swin_base_patch244_window877_kinetics400_1k_tf.tar.gz.
Preparing model: swin_base_patch244_window877_kinetics400_22k_tf.
Copying from tf_models/swin_base_patch244_windo

Mount Google Driver and Copy Weights' Files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r "/content/tars" "/content/drive/MyDrive/TF_Models/tars"